# Cherry Picking in a Decentralized Hayekian Market described through Swam Theory: First Model

In this first model, we have three different buyers, each choosing on different conditions depending on the type they belong. In particular we have:

* Type of buyer $B_1$, who always chooses the seller offering the highest quality product.
* Type of buyer $B_2$, who always chooses the seller with the highest quality-price ratio.
* Type of buyer $B_3$, who always chooses the seller with the lowest price.

## Lybraries
Let's import all the useful libraries

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import random
import statistics as st

## Parameters

Let's define the parameters shaping the dynamics

In [ ]:
#Number of agents in the model
N = 10       #numbers of sellers
M = 50       #numbers of buyers
ratio = N/M  #ratio sellers/buyers #pt in fig 1 del paper capisco che il ratio è M/N


#Parameters for the model
#micro-micro interaction
eta_0 = 1       #intensity of rate of micro-micro interaction
alpha = 1       #intensity of micro-micro interaction for sellers
rho = 2         #intensity of exponential argument in the rate of micro-micro interaction for seller
beta = 0.1      #intensity of micro-micro interaction for buyers
#macro-micro interaction
gamma_s = 0.01  #intensity of macro-micro interaction for sellers
mu_0 = 1        #intensity of rate of macro-micro interaction for sellers


#Number of cycles
n = 50000

## Quantities

Let's initialize the arrays containing the quantities defining the dynamics and define their initial conditions. 

In particular, defining two dimensional arrays we will have:

* 1rst dimension with numbers of timesteps
* 2nd dimension with number of buyers or sellers

Initial prices will be real numbers set in the interval $[1000,1005]$ (to avoid the passing through 0) and initial velocities will all be set equal to zero, both for buyers and sellers. Qualities of sellers' products will be integer number in the interval $[1000,1005]$ (to make them comparable with prices in the order of magnitude).

In [ ]:
#PRICES
b = np.full([n,M], 0.0) #buyers prices during time-steps
s = np.full([n,N], 0.0) #sellers prices during time-steps


#VELOCITIES of prices
x = np.full([n,M], 0.0) #buyers prices velocities during time-steps
y = np.full([n,N], 0.0) #sellers prices velocities during time-steps


#MEAN prices
MeanPrice_B = np.full([n,1], 0.0) #buyers mean prices during time-steps
MeanPrice_S = np.full([n,1], 0.0) #sellers mean prices during time-steps


#VARIANCE of prices
VariancePrice_S = np.full([n,1], 0.0) #buyers variance of prices during time-steps
VariancePrice_B = np.full([n,1], 0.0) #sellers variance of prices during time-steps


#PARETO MARKET EFFICIENCY
efficiency_S = np.full([n,1], 0.0)   #total buyers efficiency during time-steps
efficiency_B = np.full([n,1], 0.0)   #total sellers efficiency during time-steps
efficiency_Tot = np.full([n,1], 0.0) #total market efficiency during time-steps


#INITIAL CONDITIONS renewed at every time-step (only for the modelling)
z_b0 = [0,0] #initial conditions of every time-step for buyers
z_s0 = [0,0] #initial conditions of every time-step for sellers




#SELLERS

#Sellers Initial Cost
s_cost = np.full(N, 0.0)

#Every Seller's offered Quality
quality = np.full(N, 0.0)

#Sellers Quality/Price Ratio
ratio_qp = np.full(N, 0.0)



#BUYERS

#Buyers Type
b_type = np.full(M, 0.0)

#Buyers Chosen Seller at every timestep
chosen_seller = np.full([n,M], 0)

#Buyer Chosen Price and Seller renewed at each timestep (only for modelling)
b_choices = [0,0]




#SETTING INITIAL CONDITIONS FOR ALL QUANTITIES

#Buyers initial Prices and Types
for i in range(0,M):
    #Price
    b[0,i] = random.random()*5 + 1000
    #Type
    b_type[i] = random.randint(1,3)
    print(b_type[i])

    
#Sellers initial Prices, Costs, Qualities, Quality/Price Ratio
for j in range(0,N):
    #Price
    s[0,j] = random.random()*5 + 1000
    #Cost
    s_cost[j] = (s[0,j])*0.1
    #Quality
    quality[j] = random.randint(1000,1005)
    print(quality[j])
    #Ratio
    ratio_qp[j] = quality[j]/s[0,j]


#Calculation of initial Mean Prices
MeanPrice_S[0] = st.mean(s[0]) #buyers initial mean prices
MeanPrice_B[0] = st.mean(b[0]) #sellers initial mean price


#Calculation of initial Variances of Prices
VariancePrice_S[0] = st.variance(s[0]) #buyers initial variance of prices
VariancePrice_B[0] = st.variance(b[0]) #sellers initial variance of prices

## Functions

Now we are going to define the functions of the dynamics both for buyers and for sellers. We will have two types of interactions for both agents:

* _Micro-micro_ interaction: between the single buyer and the single seller
* _Macro-micro_ interaction: between the single agent and all the FS to which it belongs

Note that, even if present in the general formulation of the dynamics, we will not consider the interaction among buyers

### Buyers

Functions defining buyers' prices adaptation (through interaction)

In [ ]:
#BUYERS FUNCTIONS 


#Micro-Micro

#Function defining rate of micro-micro interaction
def eta_b(bb,ss,ratio,eta_0):
    if -0.01 < bb < 0.01:
        argument = (np.abs((ss-bb)/0.01))
    else:
        argument = (np.abs((ss-bb)/bb))
    xx = -(ratio)*(argument)
    etab = eta_0 * (np.exp(xx))
    return etab

#Function defining micro-micro interaction
def fi_b(bb,ss,beta):
    fib = beta * (ss-bb)
    return fib


#Function defining the seller with which the buyer will interact
def choice(s,b_type,quality,ratio_qp):
    chosen_price = s[0]
    b_choice = 0
    q = quality[0]
    value = ratio_qp[0]
    for j in range(0,N-1):
    #depending on the buyer's type, best seller (with its offered price) is chosen
        if b_type == 1.0: #B1
            if quality[j+1] > q:
                chosen_price = s[j+1]
                b_choice = j+1
                q = quality[j+1]
            elif (quality[j+1] == q):
                b_choice = random.choice([b_choice, j+1])
                chosen_price = s[b_choice]
        elif b_type == 2.0: #B2
            if ratio_qp[j+1] > value:
                chosen_price = s[j+1]
                b_choice = j+1
                value = ratio_qp[j+1]
            elif (ratio_qp[j+1] == value):
                b_choice = random.choice([b_choice, j+1])
                chosen_price = s[b_choice]
        elif b_type == 3.0: #B3
            if chosen_price > s[j+1]:
                chosen_price = s[j+1]
                b_choice = j+1
            elif (chosen_price == s[j+1]):
                b_choice = random.choice([b_choice, j+1])
    #returns price of the chosen seller and the chosen seller in this order
    return chosen_price,b_choice




#Whole Dynamics

#Function defining the whole dynamics of buyers' acceleration in ODE
def model_b(z_b,_,min_price,ratio,beta,N,eta_0):
    b = z_b[0]
    x = z_b[1]
    sumx = (eta_b(b,min_price,ratio,eta_0)) * fi_b(b,min_price,beta)
    dbdt = x
    dxdt = (1/N) * sumx
    return [dbdt,dxdt]

### Sellers

Functions defining sellers' prices adaptation (through interaction)

In [ ]:
#SELLERS FUNCTIONS


#Micro-Micro

#Function defining rate of micro-micro interaction
def eta_s(s,ratio,eta_0,rho):
    xx = -(rho/ratio)*np.abs(s)
    etas = eta_0 * (np.exp(xx))
    return etas

#Function defining micro-micro interaction
def fi_s(b,s,alpha,chosen_s,s_counter):
    if (chosen_s == s_counter): #here seller knows if it is the chosen seller for buyer i
        if (b<s):
            sig = -1
        else:
            sig = 1
    else:
        sig = -1
    fis = alpha*np.abs(s)*sig
    return fis



#Macro-Micro

#Function defining macro-micro interaction
def psi_s(s,E1,gamma_s):
    psis = gamma_s*(E1 - s)
    return psis




#Whole Dynamics

#Function defining the whole dynamics of sellers' acceleration in ODE
def model_s(z_s,_,b,E1,mu_0,eta_0,alpha,M,gamma_s,chosen_s,s_counter,rho):
    s = z_s[0]
    y = z_s[1]
    sumy = 0.0 #sum on buyers (interaction of the seller j with all buyers)
    for i in range(0,M):
        sumy = sumy + (eta_s(s,ratio,eta_0,rho)*fi_s(b[i],s,alpha,chosen_s[i],s_counter))
    dsdt = y
    dydt = (1/M)*(sumy) + mu_0 * psi_s(s,E1,gamma_s)
    return [dsdt,dydt]

## Whole Dynamics

Now we are going to solve computationally the ODEs, obtaining prices, velocities, mean prices, variances and efficiencies both for buyers and sellers

In [ ]:
tLeft = 0.1 #to define time interval


#SOLVING ODE

#iterating on Timesteps
for k in range(0,n-1):
    TimeInterval = [0,tLeft]
    #calculating first order moment of sellers prices at timestep k-1
    sumE1 = 0.0 
    for jj in range(0,N):
        sumE1 = sumE1 + s[k,jj]
    E1 = sumE1/N

#iterating on Buyers
    for i in range(0,M):
        b_choices = choice(s[k],b_type[i],quality,ratio_qp) #every buyer chooses its best seller
        chosen_price = b_choices[0]
        chosen_seller[k,i] = b_choices[1]
        z_b0 = [b[k,i],x[k,i]] #initial conditions at time-step k+1
        z_b = odeint(model_b,z_b0,TimeInterval,args=(chosen_price,ratio,beta,N,eta_0)) #solving ODEs for buyer i
        b[k+1,i] = z_b[1,0] #price at time-step k+1 of i-buyer
        x[k+1,i] = z_b[1,1] #velocity at time-step k+1 of i-buyer
        if k > 0:
        #calculating buyers' and sellers' efficiency
            if b[k,i] >= chosen_price:
                efficiency_B[k] = efficiency_B[k] + (b[k,i] - chosen_price)
                efficiency_S[k] = efficiency_S[k] + (chosen_price - s_cost[chosen_seller[k,i]])
    #calculating mean and variance of buyers' prices and total efficiency for each TimeStep
    MeanPrice_B[k+1] = st.mean(b[k+1]) 
    VariancePrice_B[k+1] = st.variance(b[k+1])
    efficiency_Tot[k] = efficiency_B[k] + efficiency_S[k]

#iterating on Sellers
    for j in range(0,N):
        seller_counter = j #to detect if j-seller is the chosen one 
        z_s0 = [s[k,j],y[k,j]] #initial conditions at time-step k+1
        z_s = odeint(model_s,z_s0,TimeInterval,args=(b[k],E1,mu_0,eta_0,alpha,M,gamma_s,chosen_seller[k],seller_counter,rho))   #solving ODEs for seller j
        s[k+1,j] = z_s[1,0] #price at time-step k+1 of j-seller
        y[k+1,j] = z_s[1,1] #velocity at time-step k+1 of j-seller
        ratio_qp[j] = (quality[j])/(s[k+1,j])
    #calculating mean and variance of sellers prices for each TimeStep
    MeanPrice_S[k+1] = st.mean(s[k+1])
    VariancePrice_S[k+1] = st.variance(s[k+1])
#to take trace of time-steps
    if (k % 1000) == 0:
        print(k)

## Plots

Note that with $\gamma = 0.01$ amplitude of buyers prices changes in time and they coordinate in three clusters depending on their type. Macro-waves appear.

Moreover, buyers Pareto market efficiency gets higher with respect to the case with $\gamma = 0.1$

In [ ]:
#BUYERS' AND SELLERS' PRICES PLOT
for i in range(0,M):
    if (b_type[i]) == 1:
        plt.plot(b[:,i],'g-') #first type buyers in green
    elif (b_type[i]) == 2:
        plt.plot(b[:,i],'m-') #second type buyers in magenta
    elif (b_type[i]) == 3:
        plt.plot(b[:,i],'y-') #third type buyers in yellow
plt.xlabel('time')            
for j in range(0,N):
        plt.plot(s[:,j],'b-') #sellers prices in blue
plt.xlabel('time')
plt.show()

In [ ]:
#SELLERS' PRICES PLOT
for j in range(0,N):
    plt.plot(s[:,j],'b-') #sellers prices in blue
plt.xlabel('time')
plt.show()

In [ ]:
#BUYERS' PRICES PLOT WITH ZOOM IN [49000,50000] TIME-STEPS INTERVAL
for i in range(0,M):
    if (b_type[i]) == 1:
        plt.plot(b[:,i],'g-') #first type buyers in green
    elif (b_type[i]) == 2:
        plt.plot(b[:,i],'m-') #second type buyers in magenta
    elif (b_type[i]) == 3:
        plt.plot(b[:,i],'y-') #third type buyers in yellow
plt.xlabel('time')
plt.xlim([49000, 50000])
plt.show()

In [ ]:
#BUYERS' PRICES VELOCITIES PLOT WITH ZOOM IN [49000,50000] TIME-STEPS INTERVAL
for i in range(0,M):
    if (b_type[i]) == 3:
        plt.plot(x[:,i],'y-') #third type buyers in yellow
    elif (b_type[i]) == 2:
        plt.plot(x[:,i],'m-') #second type buyers in magenta
    elif (b_type[i]) == 1:
        plt.plot(x[:,i],'g-') #first type buyers in green
plt.xlabel('time')
plt.ylabel('Buyers Velocities')
plt.xlim([49000, 50000])
plt.show()

In [ ]:
#MEAN SELLERS' PRICES PLOT
plt.plot(MeanPrice_S,'b')
plt.xlabel('time')
plt.ylabel('Mean Sellers Prices')
plt.show()

In [ ]:
#MEAN BUYERS' PRICES PLOT
plt.plot(MeanPrice_B,'r')
plt.xlabel('time')
plt.ylabel('Mean Buyers Prices')
plt.show()

In [ ]:
#VARIANCE OF SELLERS' PRICES PLOT
plt.plot(VariancePrice_S,'b')
plt.xlabel('time')
plt.ylabel('Variance of Sellers Prices')
plt.show()

In [ ]:
#VARIANCE OF BUYERS PRICES PLOT
plt.plot(VariancePrice_B,'r')
plt.xlabel('time')
plt.ylabel('Variance of Buyers Prices')
plt.show()

In [ ]:
#ALL EFFICIENCIES PLOT
plt.plot(efficiency_S,'b')   #sellers' efficiency in blue
plt.plot(efficiency_B,'r')   #buyers' efficiency in red
plt.plot(efficiency_Tot,'m') #total efficiency in magenta
plt.show()

In [ ]:
#BUYERS' EFFICIENCY PLOT
plt.plot(efficiency_B,'r')
plt.show()